In [1]:
!pip install rank_bm25
!pip install underthesea
!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 59.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.2 MB/s eta 0:00:00


In [2]:
import gdown

gdown.download("https://drive.google.com/uc?id=1wy8i6qAVWgvD4vzyyY7_3_0aj0J4gz5z", "/kaggle/working/test_data.csv", quiet=False)
gdown.download("https://drive.google.com/uc?id=1391dilnIYwe0uGRSJ7MwqrTUvoFPhgMT", "/kaggle/working/bm25_model.pkl", quiet=False)
print("File downloaded successfully!")

Downloading...
From: https://drive.google.com/uc?id=1wy8i6qAVWgvD4vzyyY7_3_0aj0J4gz5z
To: /kaggle/working/test_data.csv
100%|██████████| 18.4M/18.4M [00:00<00:00, 71.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1391dilnIYwe0uGRSJ7MwqrTUvoFPhgMT
From (redirected): https://drive.google.com/uc?id=1391dilnIYwe0uGRSJ7MwqrTUvoFPhgMT&confirm=t&uuid=8683843f-327f-4fdf-ac58-fb5108344dfc
To: /kaggle/working/bm25_model.pkl
100%|██████████| 232M/232M [00:02<00:00, 104MB/s]  


File downloaded successfully!


In [ ]:
import pandas as pd

data = pd.read_csv('/kaggle/working/test_data.csv')
questions = data['question'].tolist()
qids = data['qid'].tolist()

In [3]:

import re
import string

def process_cid(s):
    s = s.strip('[]')
    elements = re.split(r'[\,\s]+', s)
    return [int(element) for element in elements if element.strip().isdigit()]

result = {}

for idx, row in data.iterrows():
    qid = row['qid']
    cid = process_cid(row['cid'])
    result[qid] = cid

In [4]:

import pickle

with open('/kaggle/working/bm25_model.pkl', 'rb') as f:
    bm25 = pickle.load(f)


In [5]:

number = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
chars = ["a", "b", "c", "d", "đ", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o"]
stop_word = number + chars + ["của", "và", "các", "có", "được", "theo", "tại", "trong", "về", 
            "hoặc", "người",  "này", "khoản", "cho", "không", "từ", "phải", 
            "ngày", "việc", "sau",  "để",  "đến", "bộ",  "với", "là", "năm", 
            "khi", "số", "trên", "khác", "đã", "thì", "thuộc", "điểm", "đồng",
            "do", "một", "bị", "vào", "lại", "ở", "nếu", "làm", "đây", 
            "như", "đó", "mà", "nơi", "”", "“"]

def remove_stopword(w):
    return w not in stop_word

def remove_punctuation(w):
    return w not in string.punctuation

def lower_case(w):
    return w.lower()

def bm25_tokenizer(text):
    tokens = word_tokenize(text)
    tokens = list(map(lower_case, tokens))
    tokens = list(filter(remove_punctuation, tokens))
    tokens = list(filter(remove_stopword, tokens))
    return tokens

In [17]:
from tqdm import tqdm
import numpy as np
from underthesea import word_tokenize

candidate_list_with_scores = []
for question in tqdm(questions, desc="Searching BM25 candidates"):
    query_tokens = bm25_tokenizer(question)
    bm25_scores = bm25.get_scores(query_tokens)
    top_candidates = np.argsort(-bm25_scores)[:20]
    
    top_candidates_with_scores = [(idx, bm25_scores[idx]) for idx in top_candidates]
    candidate_list_with_scores.append(top_candidates_with_scores)

np.save("candidate_list_with_scores.npy", np.array(candidate_list_with_scores, dtype=object))
print("Complete")


Searching BM25 candidates: 100%|██████████| 10/10 [00:19<00:00,  1.98s/it]

[array([ 41204,  77626, 258194, 258187,  77625,  41205, 258174, 258173,
       258212, 258202,  41202,  41209, 223257,  71054, 258200, 258190,
       258193, 283097, 258175, 223248]), array([108382,  95838, 172222, 207691, 119852, 196001, 147521, 126144,
        85026,  76534,  82694,  42644,  69800, 151242,  22014, 222558,
       301120, 289104, 141256, 222565]), array([140057,  81565, 108185,  75749, 291947, 106254, 112310, 112311,
       149275, 194147, 116622,  99302, 100315,  94335, 314038, 123403,
       277172, 106189, 119409,  24068]), array([ 81305,  79021,   5529,   5521,  81993,   5522,  77313,   5520,
        77248,  95891,  70737, 126835,  95890, 292693,  76621,   3833,
        81992,   5523,  91390,  77246]), array([ 45968,  45970, 252656, 154631,  52966,  52965,  52963, 252659,
        45973, 106696,  77918,  75697, 250135,  45967, 139667, 252658,
       251888,  52969, 304016,  38399]), array([ 44174, 285798,  32007,  75833, 247866,  76888, 123466,  37690,
       101602